In [1]:
# 导入包
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data.dataloader import DataLoader

In [2]:
# 构建 transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])

In [3]:
#下载/加载数据集
trainset = datasets.FashionMNIST(root='./FashionMNIST', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./FashionMNIST', train=False, download=True, transform=transform)

In [4]:
BATCH_SIZE = 256
train_loader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = True)

In [5]:
class NeuralNetwork(nn.Module):  
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 默认将第0维保留下来，其余拍成一维
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),)
        
    def forward(self, x):
        x = self.flatten(x)
        # x = x.view(-1,28*28)
        logits = self.linear_relu_stack(x)
        return logits
FCN = NeuralNetwork()
print(FCN)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# 定义优化器和损失函数
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(FCN.parameters(), lr=learning_rate)

In [7]:
# 训练
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss = 0.0
    for i, (datas, labels) in enumerate(train_loader):
        # torch.Size([256, 1, 28, 28])—>datas
        # 梯度置零
        optimizer.zero_grad()
        # 训练
        # torch.Size([256, 10])—>outputs
        outputs = FCN(datas)
        # 计算损失
        loss = loss_function(outputs, labels)
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step() 
        train_loss += loss.item()
    print("Epoch :%d , Loss : %.3f"%(epoch+1, train_loss/len(train_loader.dataset)))

Epoch :1 , Loss : 0.002
Epoch :2 , Loss : 0.001
Epoch :3 , Loss : 0.001
Epoch :4 , Loss : 0.001
Epoch :5 , Loss : 0.001
Epoch :6 , Loss : 0.001
Epoch :7 , Loss : 0.001
Epoch :8 , Loss : 0.001
Epoch :9 , Loss : 0.001
Epoch :10 , Loss : 0.001


In [8]:
# 测试
correct = 0
total = 0
with torch.no_grad():
    for i , (datas, labels) in enumerate(test_loader):
        outputs = FCN(datas)
        #print(len(outputs))
        _, predicted = torch.max(outputs.data, dim=1) # 第一个是值的张量，第二个是序号的张量
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print(total)
    print("Total Accuracy：{:.3f}%".format(correct / total * 100))

10000
Total Accuracy：88.400%


# 补充

## 截取部分数据样本

In [9]:
trainset = list(trainset)[:4000]
testset = list(testset)[:1000]

In [10]:
from collections import Counter
train_label_count = []
test_label_count = []
for i in range(len(trainset)):
    train_label_count.append(trainset[i][1])
print(Counter(train_label_count))

for i in range(len(testset)):
    test_label_count.append(testset[i][1])
print(Counter(test_label_count))

Counter({1: 440, 7: 413, 3: 409, 2: 404, 6: 400, 9: 395, 4: 395, 5: 391, 8: 380, 0: 373})
Counter({4: 115, 2: 111, 0: 107, 1: 105, 6: 97, 9: 95, 7: 95, 8: 95, 3: 93, 5: 87})


## 利用GPU训练

In [ ]:
device = 'cuda'
# 创建模型
net = FCN().to(device)

In [ ]:
datas, labels = datas.to(device), labels.to(device)